In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

data_path = '/data/examples/pm25/'

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [2]:
train = pd.read_csv(data_path + "PM25_train.csv")
test = pd.read_csv(data_path + "PM25_test.csv")

print(train.shape, test.shape)
train.head()

(1116554, 10) (41223, 9)


,device_id,Date,Time,PM2.5,PM10,PM1,Temperature,Humidity,lat,lon
0,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
1,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
2,28C2DDDD415C,2017-01-01,08:03:09,21.0,0.0,0.0,24.12,83.0,23.741,120.755
3,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755
4,28C2DDDD415C,2017-01-01,08:09:04,20.0,0.0,0.0,24.12,82.0,23.741,120.755


In [3]:
#remove outliers

def removeOutliers(dataFrame, column):
    print('old', dataFrame.shape)

    devices = dataFrame['device_id'].unique()

    newDf = pd.DataFrame()

    for device in devices:
        deviceDf = dataFrame[dataFrame['device_id']==device]
        deviceDf = deviceDf[np.abs(deviceDf[column]-deviceDf[column].mean()) <= (3*deviceDf[column].std())]
        #print(device,deviceDf.shape)
        newDf = pd.concat([deviceDf, newDf])  

    newDf = newDf.reset_index(drop=True)

    print('new', newDf.shape)
    return newDf


In [4]:
train = removeOutliers(train, 'PM2.5')
#train = removeOutliers(train, 'PM10')
#train = removeOutliers(train, 'PM1')

#test = removeOutliers(test, 'PM10')
#test = removeOutliers(test, 'PM1')
#test = removeOutliers(test)


old (1116554, 10)
new (1109991, 10)


In [5]:
#train = train.groupby(by=['device_id','Date'], as_index=False).mean()
train=train.groupby(['device_id','Date'])['PM2.5','PM10','PM1','Temperature','Humidity'].mean().reset_index()
test=test.groupby(['device_id','Date'])['PM10','PM1','Temperature','Humidity'].mean().reset_index()
train.head()

,device_id,Date,PM2.5,PM10,PM1,Temperature,Humidity
0,28C2DDDD415C,2017-01-01,17.197279,0.0,0.0,26.624762,73.476190
1,28C2DDDD415C,2017-01-02,33.263158,0.0,0.0,25.061421,81.178947
2,28C2DDDD415C,2017-01-03,27.547945,0.0,0.0,24.579315,93.993151
3,28C2DDDD415C,2017-01-04,17.487310,0.0,0.0,26.476548,85.685279
4,28C2DDDD415C,2017-01-05,50.668508,0.0,0.0,25.949945,84.265193


In [6]:
#y = np.log1p(train['PM2.5'])
y = train['PM2.5']
print(y.shape)
y.head()

(7213,)


0    17.197279
1    33.263158
2    27.547945
3    17.487310
4    50.668508
Name: PM2.5, dtype: float64

In [7]:
testIds = test['device_id']

train=train.drop(columns=['PM2.5'])
#test.insert(loc=3, column='PM2.5', value=0.0)
test.head()


,device_id,Date,PM10,PM1,Temperature,Humidity
0,28C2DDDD415C,2017-01-31,0.000000,0.000000,19.915289,88.488889
1,28C2DDDD47BC,2017-01-31,0.000000,0.000000,20.045912,84.408840
2,74DA3895C1F8,2017-01-31,18.702703,12.317568,22.770068,69.763514
3,74DA3895C1FA,2017-01-31,15.787037,10.712963,20.690185,76.685185
4,74DA3895C200,2017-01-31,18.721311,10.885246,21.660410,72.221311


In [8]:
all_data = pd.concat((train.loc[:,],
                      test.loc[:,]))
print(all_data.shape)

(7465, 6)


In [9]:

#all_data=all_data.drop(columns=['Temperature','Humidity','lat','lon'])
#all_data=all_data.drop(columns=['PM10','PM1'])
#all_data.head()

In [10]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7465 entries, 0 to 251
Data columns (total 6 columns):
device_id      7465 non-null object
Date           7465 non-null object
PM10           7465 non-null float64
PM1            7465 non-null float64
Temperature    7465 non-null float64
Humidity       7465 non-null float64
dtypes: float64(4), object(2)
memory usage: 408.2+ KB


In [11]:
#all_data['Time'] = all_data['Time'].apply(lambda x: x[:2]) #compute skewness

all_data.head()


,device_id,Date,PM10,PM1,Temperature,Humidity
0,28C2DDDD415C,2017-01-01,0.0,0.0,26.624762,73.476190
1,28C2DDDD415C,2017-01-02,0.0,0.0,25.061421,81.178947
2,28C2DDDD415C,2017-01-03,0.0,0.0,24.579315,93.993151
3,28C2DDDD415C,2017-01-04,0.0,0.0,26.476548,85.685279
4,28C2DDDD415C,2017-01-05,0.0,0.0,25.949945,84.265193


In [12]:

all_data['Date'] = all_data['Date'].astype('category').cat.codes
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7465 entries, 0 to 251
Data columns (total 6 columns):
device_id      7465 non-null object
Date           7465 non-null int8
PM10           7465 non-null float64
PM1            7465 non-null float64
Temperature    7465 non-null float64
Humidity       7465 non-null float64
dtypes: float64(4), int8(1), object(1)
memory usage: 357.2+ KB


In [13]:
print(all_data.shape)
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(7465, 6)
(7465, 257)


In [14]:
all_data.head()

,Date,PM10,PM1,Temperature,Humidity,device_id_28C2DDDD415C,device_id_28C2DDDD47BC,device_id_74DA3895C1F8,device_id_74DA3895C1FA,device_id_74DA3895C200,...,device_id_74DA3895E0A4,device_id_74DA3895E0A8,device_id_74DA3895E0AC,device_id_74DA3895E0AE,device_id_74DA3895E0B0,device_id_74DA3895E0BE,device_id_74DA3895E0D0,device_id_74DA3895E0DC,device_id_74DA3895E0E0,device_id_74DA3895E0E2
0,0,0.0,0.0,26.624762,73.476190,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,0.0,25.061421,81.178947,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.0,0.0,24.579315,93.993151,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.0,0.0,26.476548,85.685279,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.0,0.0,25.949945,84.265193,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7465 entries, 0 to 251
Columns: 257 entries, Date to device_id_74DA3895E0E2
dtypes: float64(4), int8(1), uint8(252)
memory usage: 2.1 MB


In [16]:
#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]


In [17]:
X_train.to_csv('pm25_train_nor.csv',index=False)

In [18]:
print(X_train.shape,X_test.shape,y.shape)

(7213, 257) (252, 257) (7213,)


In [19]:
X_train.head()

,Date,PM10,PM1,Temperature,Humidity,device_id_28C2DDDD415C,device_id_28C2DDDD47BC,device_id_74DA3895C1F8,device_id_74DA3895C1FA,device_id_74DA3895C200,...,device_id_74DA3895E0A4,device_id_74DA3895E0A8,device_id_74DA3895E0AC,device_id_74DA3895E0AE,device_id_74DA3895E0B0,device_id_74DA3895E0BE,device_id_74DA3895E0D0,device_id_74DA3895E0DC,device_id_74DA3895E0E0,device_id_74DA3895E0E2
0,0,0.0,0.0,26.624762,73.476190,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,0.0,25.061421,81.178947,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.0,0.0,24.579315,93.993151,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0.0,0.0,26.476548,85.685279,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.0,0.0,25.949945,84.265193,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_test.head()

,Date,PM10,PM1,Temperature,Humidity,device_id_28C2DDDD415C,device_id_28C2DDDD47BC,device_id_74DA3895C1F8,device_id_74DA3895C1FA,device_id_74DA3895C200,...,device_id_74DA3895E0A4,device_id_74DA3895E0A8,device_id_74DA3895E0AC,device_id_74DA3895E0AE,device_id_74DA3895E0B0,device_id_74DA3895E0BE,device_id_74DA3895E0D0,device_id_74DA3895E0DC,device_id_74DA3895E0E0,device_id_74DA3895E0E2
0,30,0.000000,0.000000,19.915289,88.488889,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,0.000000,0.000000,20.045912,84.408840,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,18.702703,12.317568,22.770068,69.763514,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30,15.787037,10.712963,20.690185,76.685185,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,30,18.721311,10.885246,21.660410,72.221311,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
isXbg = False
if isXbg:        
    dtrain = xgb.DMatrix(X_train, label = y)
    dtest = xgb.DMatrix(X_test)

    params = {"max_depth":2, "eta":0.1}
    #cvresult = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)


    #print('Best number of trees = {}'.format(cvresult.shape[0]))
    #cvresult

    #cvresult.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()
    model = xgb.XGBRegressor() #the params were tuned using xgb.cv
    #model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)
    model.fit(X_train, y, eval_metric='rmse')
else:
    model = LassoCV(alphas = [1, 0.1, 0.001, 0.0005, 0.0007]).fit(X_train, y)
    coef = pd.Series(model.coef_, index = X_train.columns)
    print('best alpha=',model.alpha_,"Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")


best alpha= 0.001 Lasso picked 135 variables and eliminated the other 122 variables


In [23]:


rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
print(rmse, rmse.mean())



[ 7.75916875  0.55870442  0.5984629   0.64546504  0.6995915 ] 2.05227851941


In [24]:
#測試mean_squared_error
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_train)
print('mean_squared_error ',np.sqrt(mean_squared_error(y, y_pred)))

#1.14789807271

mean_squared_error  1.14789807271


In [25]:
#測試mean_squared_error
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_train)
print('mean_squared_error ',np.sqrt(mean_squared_error(np.expm1(y), np.expm1(y_pred))))

mean_squared_error  1.03039291059e+36


In [26]:
y_pred = model.predict(X_test)

out = pd.DataFrame()
out['device_id'] = testIds
out['pred_pm25'] =  y_pred
#out['pred_pm25'] =  np.expm1(y_pred)
print(out.shape)


(252, 2)


In [27]:
#out = out.groupby(by=['device_id'], as_index=False).mean()


In [28]:
out.head()

,device_id,pred_pm25
0,28C2DDDD415C,33.339510
1,28C2DDDD47BC,38.841814
2,74DA3895C1F8,16.510056
3,74DA3895C1FA,14.403241
4,74DA3895C200,15.966442


In [29]:
out.shape

(252, 2)

In [30]:
out.to_csv('pm25_submit.csv',index=False)